## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-20-10-55-15 +0000,bbc,Tory kicked out of party for 'talking to Reform',https://www.bbc.com/news/articles/c0erdwrp972o...
1,2026-01-20-10-54-00 +0000,wsj,GSK to Acquire RAPT for $2.2 Billion to Bolste...,https://www.wsj.com/business/deals/gsk-to-acqu...
2,2026-01-20-10-53-35 +0000,nyt,Trump Calls Britain’s Chagos Deal ‘Act of Grea...,https://www.nytimes.com/2026/01/20/us/politics...
3,2026-01-20-10-52-19 +0000,bbc,Wage growth slows as number of people employed...,https://www.bbc.com/news/articles/cddgrg87ly5o...
4,2026-01-20-10-48-00 +0000,wsj,Market Reaction to Greenland Tensions Has Been...,https://www.wsj.com/articles/market-reaction-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,78
32,greenland,31
583,new,13
136,he,13
235,tariffs,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...,165
7,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,156
284,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...,155
69,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/elections/why-an-...,145
11,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...,143


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,165,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...
14,57,2026-01-20-10-12-51 +0000,wapo,Death toll from high-speed train collision in ...,https://www.washingtonpost.com/world/2026/01/1...
11,54,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
99,45,2026-01-20-01-54-23 +0000,wapo,Anti-ICE protest at Minnesota church is latest...,https://www.washingtonpost.com/immigration/202...
274,41,2026-01-19-13-00-00 +0000,wsj,Half of U.S. voters say the Trump administrati...,https://www.wsj.com/politics/policy/voters-ske...
159,35,2026-01-19-22-58-10 +0000,nypost,"LI school board boss, who launched fight to sa...",https://nypost.com/2026/01/19/us-news/li-schoo...
235,34,2026-01-19-16-38-11 +0000,cbc,Investigators find broken joint on track at de...,https://www.cbc.ca/news/world/spain-train-dera...
90,31,2026-01-20-03-15-28 +0000,nypost,"40-year-old stabbed, officer assaulted after b...",https://nypost.com/2026/01/19/us-news/brawl-br...
271,31,2026-01-19-13-17-00 +0000,wsj,European Stock Markets Tumble on Trump’s Tarif...,https://www.wsj.com/finance/stocks/global-equi...
118,31,2026-01-20-01-07-39 +0000,nypost,Trump cheered by CFP national championship gam...,https://nypost.com/2026/01/19/us-news/trump-ch...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
